# 2-D Diffusionsgleichung (Stationär)

## Einführung

Unter der Annahme, dass sich die Temperatur nicht mehr mit der Zeit ändert, also das Temperaturfeld stationär ist, vereinfacht sich die Wärmeleitgleichung in eine reine Diffusionsgleichung in der Form einer sogenannte Poissongleichung (nur radiogene Elemente):

$$
0 = \frac{\partial}{\partial{x}} \left(k_{x} \frac{\partial{T}}{\partial{x}}\right) + \frac{\partial}{\partial{y}} \left(k_{y} \frac{\partial{T}}{\partial{y}}\right) + \rho H_r, \tag{1}
$$

wobei $\rho$ die Dichte, $k_{x,y}$ die Wärmeleitfähigkeit in x- und y-Richtung, und $H_r$ die radiogene Wärmeproduktion pro Masse [W/kg] ist. Vereinfachen wir nun die Gleichung noch ein wenig und nehmen an, dass die thermischen Parameter (hier, vor allem die Wärmeleitfähigkeit $k$) **isotrop** und **konstant** sind:

$$
0 = \left( \frac{\partial^2{T}}{\partial{x}^2} + \frac{\partial^2{T}}{\partial{y}^2} \right) + \frac{Q}{k}, \tag{2}
$$

wobei $Q = \rho H_r$ die Wärmeproduktionsrate pro Volumen [W/m^3] ist. 

## Das Problem
Nehmen wir an, dass die Gleichung $(2)$ in einem 2D rechteckigen Gebiet mit der Seitenlänge ($L$) 4000 m und der Tiefe ($H$) 2000 m definiert sei. In dem Gebiet sei ein Körper mit hoher Wärmeproduktionsrate eingelagert, der die Eckkoordinaten (x, z) in km habe: (1900, 900), (2100, 900), (2100, 1100) und (1900, 1100). Die Wärmeproduktionsrate $\rho H$ betrage 0.3 W/m<sup>3</sup> in dem Körper und 0 außerhalb. Die Wärmeleitfähigkeit ($k$) betrage 6.5 W/(K m). Als Randbedingung (Dirichlet) sei T = 0 °C auf dem gesamten Rand zu setzen.

<img src="./Figures/Exercise04_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau

Die beschriebene Konfiguration beschreibt in etwa die Situation hochradioaktiver Endlagerung in einem Salzstock. Dort sollen radioaktive Behälter mit einem Durchmesser von 1 m in 250 m tiefen Bohrlöchern ca. 1 km unter Tage gelagert. Der mittlere Abstand solcher Bohrlöcher beträgt ca. 50 m auf einer Breite von 250 m und einer Länge von 1 - 2 km. Mit einer Wärmeproduktionsrate von etwas weniger als 1 kW für ein 1 m hohen Behälter kommt man etwa auf die obige mittlere Wärmeproduktionrate im gesamten Ablagerungsbereich.

Um das Problem mit Hilfe von Julia lösen zu können, müssen wir erst die nötigen Module (```ExtendableSparse, Plots```) und Submodule (```GeoModBox.HeatEquation.TwoD```) definieren: 

In [1]:
using ?

Nun definieren wir die Model ($L$,$H$) und physikalischen Parameter ($k$,$Q$):

In [ ]:
# Physikalischer Parameter ---------------------------------------------- #
P       = ( 
    L       =   ?,      #   [m]
    H       =   ?,      #   [m]
    k       =   ?,        #   Waermeleitfaehigkeit, W/m/K
    # Definiere die Region der Waermequelle
    Wcave   =   ?,      # Breite der Region [ m ]
    Hcave   =   ?,      # Mächtigkeit [ m ]
    Dcave   =   ?,      # Tiefe des Zentrums [ m ]
    Xcave   =   ?,      # x-Position des Zentrum [ m ]
    Q       =   ?         # Volumetrische Waermeproduktionsrate [ W/m³ ]; Q = rho*H
)
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anzahl der Gitterpunkte und die Gitterabstände:

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   ?,        # Gitterpunkte in x-Richtung, Spalten
    y       =   ?         # Gitterpunkte in y-Richtung, Zeilen    
    
)
Δ       = (
    x       =   ?,      #   Gitterabstand in x-Richtung
    y       =   ?       #   Gitterabstand in y-Richtung
)
# ----------------------------------------------------------------------- #

Mit deren Hilfe lässt sich das numerische Gitter bestimmen, so wie die Anfangsbedingungen unseres Problems: 

In [4]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(?),
)
y       = (
    c       =   LinRange(?),
)
# ----------------------------------------------------------------------- #
# Erstellung des Anfangsbedingung --------------------------------------- #
D       = ( 
    Q       =   zeros(NC...),
    T       =   zeros(NC...),
)

# Defniere die Region of additional heat source
for i = 1:NC.x, j = 1:NC.y
    ?
end
# ----------------------------------------------------------------------- #

## Die Lösung 

### Diskretisierung

Um das Problem numerisch lösen zu können, müssen wir unsere Modeldomaine in ein numerisches Gitter unterteilen. Dabei nehmen wir an, das die Temperatur auf sogenannten *zentralen* Gitterpunkten definiert ist (siehe Abb. 2). Zur Lösung unseres Problems benutzen wir auch sogenannte *Ghost Nodes* in unserem Gitter, welche eine korrekte Einbindung der Randbedingungen ermöglichen. 

#### Gitter und Indizierung

<img src="./Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Abb. 2.** Versetztes Gitternetz. 

Das gegebene versetzte Gitter ermöglicht eine sogenannte *konservative* finite Differenzen Approximation, wobei angenommen wird, dass der Wärmefluss $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ auf dem Mittelpunkt der Gitterlinien und die Temperatur im Zentrum einer Gitterzelle definiert ist (streng genommen ist die Wärmeleitfähigkeit dann auch auf den Gitterlinien definiert; da diese allerdings konstant ist, müssen wir das hier nicht berücksichtigen; bei variablen thermischen Parameter muss die Gleichung etwas anders diskreditiert werden). Außerdem können wir durch die zentralen Temperaturgitterpunkte, in Verbindung mit den *Ghost Nodes* relativ einfach Randbedingungen einbinden, die die gleiche Fehlerordnung besitzen, wie die zentralen Differenzenquotienten im Inneren unseres Models. 

Bei der Indizierung unserer Gitterpunkte unterscheiden wir zwischen *lokalen* und *globalen* Indizes. Der lokale Index beschreibt die Position auf dem *i*,*j*-Gitter. Der globale Index, ist ein durchlaufender Index von 1 bis nx*ny und entspricht der Anzahl der Gleichungen, d.h. die Gesamtanzahl der inneren Gitterpunkte. Der globale Index wird auch beim Aufstellen der Koeffizientenmatrix für unser lineares Gleichungssystem verwendet. 

Für jeden Gitterpunkt, also für jede Gleichung, gibt ein sogenannter numerischer Stempel (*stencil*) die Position der Gitterpunkte an, welche für die jeweilige Gleichung gültig sind. Die Koeffizienten für diese Gitterpunkte sind dann jeweils ungleich null und alle anderen gleich null. Die Nomenklatur für den Stempel richtet sich häufig nach der eines Kompases, d.h. wir besitzen Punkte auf: Süden, Westen, Zentral, Osten, und Norden. Für jede Gleichung, ist der globale Index ($ii$) jedes Punktes des Stempels gegeben durch die relative Position zum Zentral Punkt des Stempels, d.h.: 

$$
iS = ii - nx,\\\
iW = ii - 1,\\\   
iZ = ii, \\\
iE = ii + 1,\\\
iN = ii + nx.
$$

#### Finite Differenzen Approximation

Nun können wir die partielle Differentialgleichung durch unsere finiten Differenzen approximieren durch: 

$$
0 = \left( \frac{T_{i-1,j} - 2T_{i,j} + T_{i+1,j}}{\Delta{x}^2} + \frac{T_{i,j-1} - 2T_{i,j} + T_{i,j+1}}{\Delta{y}^2} \right) + \frac{Q}{k}, \tag{3}
$$

wobei $i, j$ jeweils die Indizes und $\Delta{x}, \Delta{y}$ die Gitterabstände in x- und y- Richtung sind. Durch Umformung erhalten wir ein lineares Gleichungsystem mit 5 Koeffizienten in der Form: 

$$
b T_{i,j-1} + aT_{i-1,j} - (2a + 2b) T_{i,j} + a T_{i+1,j} + b T_{i,j+1} = - \frac{Q}{k}, \tag{4}
$$

wobei $ a = 1 / \Delta{x}^2$ und $b = 1 / \Delta{y}^2$ ist. 

### Randbedingungen

Die Temperatur auf den *Ghost Nodes* ist für *Dirichlet* und *Neumann* Randbedingungen genau so definiert, wie im 1-D Fall der [expliziten](./02_1D_Heat_explicit.ipynb) oder [impliziten](./03_1D_Heat_implicit.ipynb) Lösung der Wärmediffusionsgleichung (diesmal für vier Ränder statt für zwei). Da wir wieder ein lineares Gleichungssystem haben, müssen wir die Koeffizienten und die rechte Seite für die Gleichungen der **inneren Gitterpunkte** in der Nähe der Ränder, in Abhängigkeit der Randbedingungen, mit Hilfe der Temperatur auf den *Ghost Nodes* wie folgt modifizieren (Herleitung siehe Vorlesung): 

**Dirichlet** <br>
*West*
$$
bT_{1,j-1} - (3a + 2b)T_{1,j} + aT_{2,j} + bT_{1,j+1} = -\frac{Q}{k} - 2aT_{BC,W}
$$
*East*
$$
bT_{nx,j-1} + aT_{nx-1,j} - (3a + 2b)T_{nx,j} + bT_{nx,j+1} = -\frac{Q}{k} - 2aT_{BC,E}
$$
*South*
$$
aT_{i-1,1} - (2a + 3b)T_{i,1} + aT_{i+1,1} + bT_{i,2} = -\frac{Q}{k} - 2bT_{BC,S}
$$
*North*
$$
bT_{i,ny-1} + aT_{i-1,ny} - (2a + 3b)T_{i,ny} + aT_{i+1,ny} = -\frac{Q}{k} - 2bT_{BC,N}
$$

**Neumann**<br>
*West*
$$
bT_{1,j-1} - (a + 2b)T_{1,j} + aT_{2,j} + bT_{1,j+1} = -\frac{Q}{k} + a c_W \Delta{x}
$$
*East*
$$
bT_{nx,j-1} + aT_{nx-1,j} - (a + 2b)T_{nx,j} + bT_{nx,j+1} = -\frac{Q}{k} - a c_E \Delta{x}
$$
*South*
$$
aT_{i-1,1} - (2a + b)T_{i,1} + aT_{i+1,1} + bT_{i,2} = -\frac{Q}{k} + b c_S \Delta{y}
$$
*North*
$$
bT_{i,ny-1} + aT_{i-1,ny} - (2a + b)T_{i,ny} + aT_{i+1,ny} = -\frac{Q}{k} -b c_N \Delta{y}
$$

Bei der Initialisierung der Randbedingungen bediehnung wir uns eines Tricks und definieren die Temperatur auf den *Ghost nodes* später im Skript: 

In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (
    type    = (W=:?, E=:?, N=:?, S=:?),
    val     = (W=:?,E=:?,N=:?,S=:?)
)
# ----------------------------------------------------------------------- #

### Lösen der Problems

Gleichung $(4)$ ist ein Lineares Gleichungsystem der Form 

$$
\bm{K} T = T_i, 
$$

mit einer Koeffizientenmatrix $\bm{K}$ mit fünf nicht-null Diagonalen, der Anfangstemperaturbedingung $T_i$, und der stationären Lösung $T$. Daher definieren wir als nächstes die Parameter für unser lineares Gleichungssystem: 

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(?)
K       =   ExtendableSparseMatrix(?,?)
rhs     =   zeros(?)
# ----------------------------------------------------------------------- #

Das Aufstellen der Koeffizientenmatrix, die Modifikation der rechten Seite wie oben beschrieben, und die Lösung des Gleichungssystems werden in der Funktion ```Poisson!()``` durchgeführt. Durch die Einbindung des Submoduls ```GeoModBox.HeatEquation.TwoD``` kann die Funktion hier direkt aufgerufen werden (Achtet dabei auf die der Funktion mitzugebenen Parameter!). 

In [ ]:
# Solve equation -------------------------------------------------------- #
Poisson2Dc!(?)
# ----------------------------------------------------------------------- #

Alternativ, könnte man auch die Rechenschritte aus der Funktion direkt hier programmieren, wobei man dann nicht mehr das Submodul laden muss. 

## Visualisierung

Zum Schluss plotten wir das Ergebniss noch: 

In [ ]:
# Plot solution --------------------------------------------------------- #
p = heatmap(?)

contour!(?)

display(p)

savefig("./Results/04_Steady_State_Solution.png")
# ----------------------------------------------------------------------- #